In [1]:
from data import load_dataset

### 1. Read data

In [2]:
train_x, train_y, test_x, test_y, classes = load_dataset()
print("number of samples in train dataset: ", train_x.shape[0])
print("number of samples in test  dataset: ", test_x.shape[0])
print("shape of train_x: ", train_x.shape)
print("shape of train_y: ", train_y.shape)
print("shape of test_x: ", test_x.shape)
print("shape of test_y: ", test_y.shape)

number of samples in train dataset:  209
number of samples in test  dataset:  50
shape of train_x:  (209, 64, 64, 3)
shape of train_y:  (1, 209)
shape of test_x:  (50, 64, 64, 3)
shape of test_y:  (1, 50)


In [3]:
train_y

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
train_x = train_x.reshape(train_x.shape[0], -1)
train_y = train_y.reshape(train_y.shape[1], -1)
test_x = test_x.reshape(test_x.shape[0], -1)
test_y = test_y.reshape(test_y.shape[1], -1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
print(train_x)

(209, 12288) (209, 1) (50, 12288) (50, 1)
[[ 17  31  56 ...   0   0   0]
 [196 192 190 ...  82  80  81]
 [ 82  71  68 ... 138 141 142]
 ...
 [143 155 165 ...  85 107 149]
 [ 22  24  23 ...   4   5   0]
 [  8  28  53 ...   0   0   0]]


### 2. Data standardization processing

In [5]:
train_x = train_x / 255.
test_x = test_x / 255.

In [6]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(train_x.shape[1], 1))
net

Sequential(
  (0): Linear(in_features=12288, out_features=1, bias=True)
)

In [7]:
net[0].weight.data.normal_(0, 0.01), net[0].bias.data.fill_(0)

(tensor([[-0.0077,  0.0064, -0.0041,  ...,  0.0098, -0.0141,  0.0054]]),
 tensor([0.]))

In [8]:
loss = nn.BCEWithLogitsLoss()
loss

BCEWithLogitsLoss()

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.001)

In [10]:
X = torch.from_numpy(train_x).float()
y = torch.from_numpy(train_y).float()
X, y.shape

(tensor([[0.0667, 0.1216, 0.2196,  ..., 0.0000, 0.0000, 0.0000],
         [0.7686, 0.7529, 0.7451,  ..., 0.3216, 0.3137, 0.3176],
         [0.3216, 0.2784, 0.2667,  ..., 0.5412, 0.5529, 0.5569],
         ...,
         [0.5608, 0.6078, 0.6471,  ..., 0.3333, 0.4196, 0.5843],
         [0.0863, 0.0941, 0.0902,  ..., 0.0157, 0.0196, 0.0000],
         [0.0314, 0.1098, 0.2078,  ..., 0.0000, 0.0000, 0.0000]]),
 torch.Size([209, 1]))

In [11]:
test_X = torch.from_numpy(test_x).float()
test_Y = torch.from_numpy(test_y).float()
test_X.shape, test_Y.shape

(torch.Size([50, 12288]), torch.Size([50, 1]))

In [12]:
num_epochs = 5000
for epoch in range(num_epochs):
    l = loss(net(X) ,y)
    trainer.zero_grad()
    l.backward()
    trainer.step()
    l = loss(net(X), y)
    if epoch % 100 == 0:
        print(f'epoch {epoch}, loss {l:f}')

epoch 0, loss 0.648607
epoch 100, loss 0.585319
epoch 200, loss 0.549666
epoch 300, loss 0.522912
epoch 400, loss 0.500968
epoch 500, loss 0.482163
epoch 600, loss 0.465610
epoch 700, loss 0.450773
epoch 800, loss 0.437302
epoch 900, loss 0.424951
epoch 1000, loss 0.413542
epoch 1100, loss 0.402937
epoch 1200, loss 0.393030
epoch 1300, loss 0.383737
epoch 1400, loss 0.374988
epoch 1500, loss 0.366725
epoch 1600, loss 0.358899
epoch 1700, loss 0.351468
epoch 1800, loss 0.344399
epoch 1900, loss 0.337659
epoch 2000, loss 0.331221
epoch 2100, loss 0.325063
epoch 2200, loss 0.319162
epoch 2300, loss 0.313501
epoch 2400, loss 0.308063
epoch 2500, loss 0.302832
epoch 2600, loss 0.297796
epoch 2700, loss 0.292942
epoch 2800, loss 0.288258
epoch 2900, loss 0.283736
epoch 3000, loss 0.279365
epoch 3100, loss 0.275137
epoch 3200, loss 0.271044
epoch 3300, loss 0.267079
epoch 3400, loss 0.263236
epoch 3500, loss 0.259508
epoch 3600, loss 0.255889
epoch 3700, loss 0.252375
epoch 3800, loss 0.24896